# Testing out Local LLMs

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [2]:
import numpy as np

In [7]:
tt = ("You are PayLLM, a conversational payment assistant. NEVER call tools until you have collected all required information:\n\n"
"You should never use external knowledge, assumptions or information beyond what is explicitly shared or recieved.\n"
"Follow this structured flow and do NOT call tools unless all required information is available:\n"
"1. Greet the user if they greet you.\n"
"2. Ask for the user their state and the service they want to pay the bill for.\n"
"3. Get the list of service providers using fetch_service_provider tool.\n"
"4. Ask for the service provider.\n"
"5. Ask for the bill number.\n"
"6. Confirm fetching the bill.\n"
"7. Display the bill amount.\n"
"8. Ask for payment confirmation.\n"
"9. Confirm payment.\n"
"Maintain the flow based on previous responses.")

' '.join(tt.split('\n'))

'You are PayLLM, a conversational payment assistant. NEVER call tools until you have collected all required information:  You should never use external knowledge, assumptions or information beyond what is explicitly shared or recieved. Follow this structured flow and do NOT call tools unless all required information is available: 1. Greet the user if they greet you. 2. Ask for the user their state and the service they want to pay the bill for. 3. Get the list of service providers using fetch_service_provider tool. 4. Ask for the service provider. 5. Ask for the bill number. 6. Confirm fetching the bill. 7. Display the bill amount. 8. Ask for payment confirmation. 9. Confirm payment. Maintain the flow based on previous responses.'

In [3]:
# Loading the model
# llm = ChatOllama(model="llama3.2:1b", temperature=0)
llm = ChatOllama(model="llama3.2", temperature=0)

In [6]:
initialSystemMessage = '''You are an excellent virtual assistant to help with payment of bills.
You should follow the following steps strictly to pay the bills:
1. Ask the user the bill number
2. Ask the user if you should fetch the bill details.
3. Inform the user the bill details once fetched.
4. Ask the user if you should pay the bill.
5. Is the user agrees, pay the bill.

You should never user external knowledge, assumptions or information beyond what is explicitly shared or recieved.
'''


messages = [
    ("system",initialSystemMessage),
    ("human", "Pay my bill")
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="To proceed with paying your bill, I need to know a few details first.\n\nCan you please tell me the number of the bill you'd like to pay?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-12T22:14:36.447587Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1133021959, 'load_duration': 29848625, 'prompt_eval_count': 127, 'prompt_eval_duration': 201000000, 'eval_count': 33, 'eval_duration': 900000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-d4d26844-08a5-4aad-9a64-013bf559dd9c-0', usage_metadata={'input_tokens': 127, 'output_tokens': 33, 'total_tokens': 160})

## Testing Tools

### Calculator Tools

In [4]:
@tool
def add(a: int, b: int) -> int:
    """Add (+) two numbers"""
    return a+b
    
@tool
def sub(a: int, b: int) -> int:
    """Subtract (-) two numbers"""
    return a-b
    
@tool
def mul(a: int, b: int) -> int:
    """Multiply (*, x) two numbers"""
    return a*b
    
@tool
def div(a: int, b: int) -> int:
    """Divide (/) two numbers"""
    return a/b

@tool
def fetchUserDets(name: str) -> str:
    """Function to fetch user details. Input is name of the user. Output relevant details."""
    # hmap = 
    pass



tools = [mul, div, add, sub]
toolsMap = {'add':add, "sub":sub, "mul":mul, "div":div}

llmTools = llm.bind_tools(tools)


In [14]:
query = "Add 3 and 4?"
query = "What is 3 multiplied 12? Also, what is 11 + 49?"

# llmTools.invoke(query).tool_calls

messages = [
    ("system","You are a helpful assistant. be kind"),
    ("human", "Tell me the answer to 3 * 12+8 * 213")
]
ai_msg = llmTools.invoke(messages)
messages.append(ai_msg)
ai_msg

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-24T23:26:50.365703Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1046341750, 'load_duration': 33897542, 'prompt_eval_count': 347, 'prompt_eval_duration': 215000000, 'eval_count': 28, 'eval_duration': 795000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-5029179c-cd05-42d2-842b-2a7f364e646f-0', tool_calls=[{'name': 'add', 'args': {'a': '3 * 12', 'b': '8 * 213'}, 'id': '2d90d4f5-9bda-497f-b61b-4514d504997c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 347, 'output_tokens': 28, 'total_tokens': 375})

In [15]:
ai_msg.tool_calls

[{'name': 'add',
  'args': {'a': '3 * 12', 'b': '8 * 213'},
  'id': '2d90d4f5-9bda-497f-b61b-4514d504997c',
  'type': 'tool_call'}]

In [16]:
messages

[('system', 'You are a helpful assistant. be kind'),
 ('human', 'Tell me the answer to 3 * 12+8 * 213'),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-24T23:26:50.365703Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1046341750, 'load_duration': 33897542, 'prompt_eval_count': 347, 'prompt_eval_duration': 215000000, 'eval_count': 28, 'eval_duration': 795000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-5029179c-cd05-42d2-842b-2a7f364e646f-0', tool_calls=[{'name': 'add', 'args': {'a': '3 * 12', 'b': '8 * 213'}, 'id': '2d90d4f5-9bda-497f-b61b-4514d504997c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 347, 'output_tokens': 28, 'total_tokens': 375})]

In [15]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("placeholder", "{msgs}") # <-- This is the changed part
])

In [18]:
prompt_template.invoke({"msgs": [HumanMessage(content="helli hela!")]})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='helli hela!', additional_kwargs={}, response_metadata={})])

In [8]:
ai_msg = llmTools.invoke(messages)

In [9]:
print(ai_msg.content)

The answer to 3 * 12 is 36.

The answer to 11 - 49 is -38.


In [10]:
ai_msg

AIMessage(content='The answer to 3 * 12 is 36.\n\nThe answer to 11 - 49 is -38.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-24T23:24:26.259054Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1108464084, 'load_duration': 31862125, 'prompt_eval_count': 142, 'prompt_eval_duration': 388000000, 'eval_count': 25, 'eval_duration': 685000000, 'message': Message(role='assistant', content='The answer to 3 * 12 is 36.\n\nThe answer to 11 - 49 is -38.', images=None, tool_calls=None)}, id='run-a1d082b4-c1cd-4821-b174-4d7676facfed-0', usage_metadata={'input_tokens': 142, 'output_tokens': 25, 'total_tokens': 167})

In [15]:
print(ai_msg.content)

The answer to 3 * 12 is 36.

The answer to 11 - 49 is -38.


---

In [2]:
import json
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables import RunnableLambda
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.memory import ConversationBufferMemory


import traceback

# Initialize the LLaMA 3.2 model
model = ChatOllama(model="llama3.2:latest", temperature=0)
output_parser = StrOutputParser()

In [19]:
from langchain.prompts import ChatPromptTemplate

from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)


In [14]:
print('''You are PayLLM, a conversational payment assistant. Follow this structured flow:
1. Greet the user if they greet you.
2. Ask for the user's state.
3. Ask for the service provider.
4. Ask for the bill number.
5. Confirm fetching the bill.
6. Display the bill amount.
7. Ask for payment confirmation.
8. Confirm payment.
Maintain the flow based on previous responses.''')

You are PayLLM, a conversational payment assistant. Follow this structured flow:
1. Greet the user if they greet you.
2. Ask for the user's state.
3. Ask for the service provider.
4. Ask for the bill number.
5. Confirm fetching the bill.
6. Display the bill amount.
7. Ask for payment confirmation.
8. Confirm payment.
Maintain the flow based on previous responses.


In [1]:
print("You are PayLLM, a conversational payment assistant. NEVER call tools until you have collected all required information:\n"
               "You should never use external knowledge, assumptions or information beyond what is explicitly shared or recieved.\n"
               "Follow this strict sequence and do NOT proceed to the next step unless all information from the previous step is available:\n"
               "1. Greet the user if they greet you.\n"
               "2. Ask for the user their state and the service they want to pay the bill for.\n"
               "3. Get the list of service providers using fetch_service_provider tool.\n"
               "4. Ask for the service provider.\n"
               "5. Ask for the bill number.\n"
               "6. Confirm fetching the bill.\n"
               "7. Display the bill amount.\n"
               "8. Ask for payment confirmation.\n"
               "9. Confirm payment.\n"
               "Maintain the flow based on previous responses.")

You are PayLLM, a conversational payment assistant. NEVER call tools until you have collected all required information:
You should never use external knowledge, assumptions or information beyond what is explicitly shared or recieved.
Follow this strict sequence and do NOT proceed to the next step unless all information from the previous step is available:
1. Greet the user if they greet you.
2. Ask for the user their state and the service they want to pay the bill for.
3. Get the list of service providers using fetch_service_provider tool.
4. Ask for the service provider.
5. Ask for the bill number.
6. Confirm fetching the bill.
7. Display the bill amount.
8. Ask for payment confirmation.
9. Confirm payment.
Maintain the flow based on previous responses.
